# Build a dataset out of all data collected

In [27]:
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from tqdm import tqdm
from pprint import pprint
import logging
# set up logger
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
# create a file handler
handler = logging.FileHandler('company_overview_dataset.log')
handler.setLevel(logging.INFO)
# create a logging format
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
# add the handlers to the logger
logger.addHandler(handler)

Convert jsons into an easy to use dict

In [28]:
overviews = {}
# iterate over all json files in the prices folder
for file in tqdm(os.listdir('overview')):
    # open the json file
    with open(os.path.join('overview', file)) as f:
        overview = json.load(f)
        overviews[overview[0]['Symbol']] = overview
# pprint(overviews)

100%|██████████| 630/630 [00:00<00:00, 1272.75it/s]


### Create the overview dataframe

Only take values that could be converted to float

In [29]:
# create a row for each stock and date
keys = ['Symbol']
rows = []
for stock, datas in tqdm(overviews.items()):
    cols = [stock]
    for key, val in datas[0].items():
        try:
            cols.append(float(val))
            if key not in keys:
                keys.append(key)
        # create an exception when the value is not a number
        except ValueError:
            pass
    rows.append(cols)
# create a dataframe from the rows
price_df = pd.DataFrame(rows, columns=keys)
price_df.to_csv('overview.csv', index=False)
price_df
# convert all columns to numeric except for the date and stock
# price_df[price_df.columns[2:]] = price_df[price_df.columns[2:]].apply(pd.to_numeric, errors='coerce')

100%|██████████| 630/630 [00:00<00:00, 52500.68it/s]


,Symbol,CIK,MarketCapitalization,EBITDA,PERatio,PEGRatio,BookValue,DividendPerShare,DividendYield,EPS,...,PriceToSalesRatioTTM,PriceToBookRatio,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding
0,A,1090872.0,4.507400e+10,1.994000e+09,36.420,2.360,17.970,0.84,0.0058,4.18,...,6.500,8.630,6.920,24.870,1.045,160.03,111.96,1.531400e+02,1.341900e+02,2.958600e+08
1,AA,1675149.0,9.205927e+09,2.003000e+09,10.120,-0.290,28.580,0.40,0.0073,5.14,...,0.583,1.485,0.630,5.120,2.463,97.48,33.47,4.851000e+01,4.943000e+01,1.769690e+08
2,AAL,6201.0,1.100886e+10,4.103000e+09,89.050,0.627,-8.910,0.00,0.0303,0.19,...,0.186,0.934,-361.050,1.540,21.420,11.65,14.60,1.463000e+01,6.506420e+08,NaN
3,AAP,1158449.0,8.987009e+09,1.012869e+09,19.440,0.909,45.860,5.50,0.0383,7.80,...,0.766,3.130,1.077,12.610,1.139,226.09,138.52,1.482800e+02,1.748400e+02,5.925400e+07
4,AAPL,320193.0,2.400657e+12,1.252880e+11,25.760,2.750,3.581,0.91,0.0060,5.89,...,5.510,44.630,5.920,17.530,1.278,178.80,124.17,1.389200e+02,1.477200e+02,1.582190e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,ZEUS,917470.0,5.315640e+08,1.817680e+08,4.929,0.460,46.060,0.29,0.0078,9.69,...,0.131,0.715,0.236,3.447,1.776,49.00,22.15,3.708000e+01,3.091000e+01,1.112990e+07
626,ZIONL,109380.0,6.500000e-02,0.000000e+00,0.000,0.000,0.000,0.00,0.0000,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
627,ZLAB,1704292.0,3.964116e+09,-5.182850e+08,11.520,0.000,0.000,-5.84,2.0550,0.00,...,1.208,61.290,20.980,37.560,36.780,97909000.00,NaN,NaN,NaN,NaN
628,ZM,1585521.0,2.325129e+10,7.648690e+08,33.420,10.950,19.690,0.00,0.0000,2.38,...,4.721,3.719,3.759,23.410,-0.265,152.69,63.55,7.123000e+01,8.846000e+01,2.456520e+08


# TODO repeat this process for all other data sheets in the jsons